 ## Using LR, NB, SVM, KNN, RF, GBT  classifier to classify texts into different categories

In [5]:
import numpy as np
from numpy import newaxis as na
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import os
import sys
import numpy as np
import shap
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

import pandas as pd
import numpy as np
from itertools import permutations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from nltk.stem import PorterStemmer
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from nltk.tokenize.treebank import TreebankWordDetokenizer
import csv 
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics import classification_report, accuracy_score

### Dataset Import

In [6]:
df=pd.read_csv('bengali_hate_v2.0.csv', sep=',', encoding='utf-8')

In [7]:
df.shape

(5698, 3)

In [30]:
train=df[:4698]
train.shape

(4698, 3)

In [31]:
test=df[4698:]
test.shape

(1000, 3)

In [32]:
test.head()

,text,label,target
4698,ছিঃ ছিঃ মিথিলা এতো রুচিহীন জঘন্য,Personal,0
4699,নাস্তিক বা হিন্দুরা আপনার এই ধরনের বক্তব্য শুন...,Religious,2
4700,তোমরা ব্যবসার জন্য ধর্মকে কেন্দ্র করে এইগুলো য...,Religious,2
4701,পুলিশের নির্বিচার গুলি বর্ষণে পাখির মত মরছে মা...,Geopolitical,3
4702,আপনারা যারা এতবাজে কমেন্ট করেছেন যে আপনারা মুস...,Religious,2


In [33]:
stop_words='stopwords-bn.txt'
text_data=[]

with open(stop_words,'r',encoding='utf-8') as temp_output_file:
    reader=csv.reader(temp_output_file, delimiter='\n')
    for row in reader:
        text_data.append(row)
stop_word_list=[x[0] for x in text_data]

In [34]:
stop_words = set(stop_word_list)  

def textCleaner(example_sent): 
    word_tokens = word_tokenize(example_sent)  
    #filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_train = TreebankWordDetokenizer().detokenize(word_tokens)

    return filtered_train

In [35]:
filtered_test = test['text'].apply(textCleaner)
filtered_train = train['text'].apply(textCleaner)

In [36]:
train['label'].value_counts()

Personal        1850
Geopolitical    1408
Religious        778
Political        662
Name: label, dtype: int64

In [37]:
x_train, y_train = filtered_train, train['target'].values
x_test, y_test = filtered_test, test['target'].values


In [38]:
tfidf_vectorizer = CountVectorizer()
x_train = tfidf_vectorizer.fit_transform(x_train)
x_test = tfidf_vectorizer.transform(x_test)

## NB Classifier

In [39]:
nb = MultinomialNB(alpha = 0.1)
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

from sklearn.metrics import matthews_corrcoef
print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.65      0.73      0.68       339
Geopolitical       0.54      0.46      0.50       152
   Religious       0.58      0.55      0.56       179
   Political       0.70      0.68      0.69       330

    accuracy                           0.64      1000
   macro avg       0.62      0.60      0.61      1000
weighted avg       0.64      0.64      0.64      1000

MCC: 0.4947396978672513


## kNN Classifier

In [40]:
knn = KNeighborsClassifier(n_neighbors=12)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

#from sklearn.metrics import matthews_corrcoef
print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.44      0.94      0.60       339
Geopolitical       0.53      0.05      0.10       152
   Religious       0.66      0.17      0.27       179
   Political       0.87      0.57      0.69       330

    accuracy                           0.55      1000
   macro avg       0.63      0.43      0.42      1000
weighted avg       0.64      0.55      0.50      1000

MCC: 0.39575848065112784


## SVM Classifier

In [41]:

# Let's run the model and get its accuracy score
from sklearn.svm import SVC
svm=SVC(kernel="linear", degree=5, probability=True) # 0.9306853582554517

svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.58      0.81      0.68       339
Geopolitical       0.53      0.32      0.40       152
   Religious       0.63      0.61      0.62       179
   Political       0.78      0.63      0.70       330

    accuracy                           0.64      1000
   macro avg       0.63      0.59      0.60      1000
weighted avg       0.65      0.64      0.63      1000

MCC: 0.4991882985833123


In [42]:
gnb = GaussianNB()
gnb.fit(x_train.toarray(), y_train)

y_pred = gnb.predict(x_test.toarray())

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.45      0.14      0.22       339
Geopolitical       0.19      0.70      0.30       152
   Religious       0.28      0.35      0.31       179
   Political       0.52      0.16      0.24       330

    accuracy                           0.27      1000
   macro avg       0.36      0.34      0.27      1000
weighted avg       0.40      0.27      0.26      1000

MCC: 0.11354356701498186


## LR Classifier

In [43]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.59      0.81      0.68       339
Geopolitical       0.52      0.33      0.40       152
   Religious       0.63      0.54      0.58       179
   Political       0.79      0.69      0.74       330

    accuracy                           0.65      1000
   macro avg       0.63      0.59      0.60      1000
weighted avg       0.65      0.65      0.64      1000

MCC: 0.510773364956144


In [44]:
dt = DecisionTreeClassifier()

dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.60      0.68      0.64       339
Geopolitical       0.47      0.38      0.42       152
   Religious       0.55      0.57      0.56       179
   Political       0.69      0.64      0.67       330

    accuracy                           0.60      1000
   macro avg       0.58      0.57      0.57      1000
weighted avg       0.60      0.60      0.60      1000

MCC: 0.4469684344554203


## RF Classifier

In [45]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.62      0.82      0.71       339
Geopolitical       0.62      0.36      0.46       152
   Religious       0.66      0.61      0.64       179
   Political       0.79      0.70      0.74       330

    accuracy                           0.68      1000
   macro avg       0.67      0.62      0.64      1000
weighted avg       0.68      0.68      0.67      1000

MCC: 0.5474998302901893


## GBT Classifier

In [46]:
gbt = GradientBoostingClassifier(random_state=0, n_estimators=200)
gbt.fit(x_train, y_train)
y_pred = gbt.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.59      0.88      0.71       339
Geopolitical       0.61      0.30      0.40       152
   Religious       0.68      0.60      0.64       179
   Political       0.84      0.67      0.74       330

    accuracy                           0.67      1000
   macro avg       0.68      0.61      0.62      1000
weighted avg       0.69      0.67      0.66      1000

MCC: 0.5494291482603195


In [47]:
ab = AdaBoostClassifier(random_state=0, n_estimators=200)
ab.fit(x_train, y_train)
y_pred = ab.predict(x_test)

categories = ['Personal', 'Geopolitical','Religious','Political']
print(classification_report(y_test, y_pred, target_names=categories))

print('MCC:', matthews_corrcoef(y_test, y_pred))

              precision    recall  f1-score   support

    Personal       0.59      0.81      0.69       339
Geopolitical       0.54      0.34      0.41       152
   Religious       0.61      0.59      0.60       179
   Political       0.81      0.65      0.72       330

    accuracy                           0.65      1000
   macro avg       0.64      0.60      0.61      1000
weighted avg       0.66      0.65      0.64      1000

MCC: 0.512066397619003
